# iris 분류
- test 비율은 20%, random_state 는 2021
- MinMaxScaler 적용
- cv=5
- 결정트리 - max_depth, min_samples_split
- 최적의 파라미터와 정확도 계산

In [35]:
import pandas as pd
import numpy as np

In [36]:
from sklearn.datasets import load_iris
iris = load_iris()

In [37]:
iris_df = pd.DataFrame(iris.data, columns=iris.feature_names)
iris_df['Species'] =  iris.target
iris_df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),Species
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [38]:
# 스케일링
from sklearn.preprocessing import MinMaxScaler
mm_scaler = MinMaxScaler()
iris_scaled = mm_scaler.fit_transform(iris.data)

In [39]:
# 스케일링 된 데이터 확인
s_df = pd.DataFrame(iris_scaled, columns = iris.feature_names)
s_df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,0.222222,0.625000,0.067797,0.041667
1,0.166667,0.416667,0.067797,0.041667
2,0.111111,0.500000,0.050847,0.041667
3,0.083333,0.458333,0.084746,0.041667
4,0.194444,0.666667,0.067797,0.041667


In [57]:
# 학습, 테스트 데이터 나누기
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(iris_scaled, iris.target, stratify = iris.target, test_size=0.2, random_state=2021)

In [41]:
# 하이퍼파라미터 튜닝
from sklearn.model_selection import GridSearchCV

In [42]:
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier(random_state=2021)

In [43]:
# parameter 를 Dictionary 형태로 설정
params = {
    'max_depth':[2,3,4,5,6],
    'min_samples_split':[2,3,4]
}

In [44]:
grid_dtc = GridSearchCV(dtc, param_grid=params, cv=5, scoring='accuracy')

In [45]:
grid_dtc.fit(X_train,y_train)  # 학습 수행

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=2021),
             param_grid={'max_depth': [2, 3, 4, 5, 6],
                         'min_samples_split': [2, 3, 4]},
             scoring='accuracy')

In [46]:
# 모델 평가
from sklearn.metrics import accuracy_score

In [47]:
# 최적 파라미터와 최고 정확도
print(grid_dtc.best_params_) 
print(grid_dtc.best_score_)

{'max_depth': 4, 'min_samples_split': 2}
0.9666666666666668


In [48]:
# 최적 예측 
pred = grid_dtc.best_estimator_.predict(X_test)
accuracy_score(y_test, pred)

0.9

### One-hot encoding을 하는 경우

In [15]:
iris.target.shape

(150,)

In [49]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder()
classes = encoder.fit_transform(iris.target.reshape(-1,1))
classes.shape    # 타겟변수인 y가 2차원이 되었음을 확인

# [ [1], [2] ,[3] ] 이런식으로 바꾸는게 reshape(-1,1)

(150, 3)

In [20]:
classes

<150x3 sparse matrix of type '<class 'numpy.float64'>'
	with 150 stored elements in Compressed Sparse Row format>

In [50]:
classes.toarray()[:5]

array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.]])

In [59]:
X_train, X_test, Y_train, Y_test = train_test_split(
    iris_scaled, classes, stratify = iris.target, test_size=0.2, random_state=2021
)

In [60]:
X_train.shape, X_test.shape, Y_train.shape, Y_test.shape

((120, 4), (30, 4), (120, 3), (30, 3))

In [53]:
Y_test[:5].toarray()

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.]])

In [54]:
dtc = DecisionTreeClassifier(random_state=2021)
grid_dtc = GridSearchCV(dtc, param_grid=params, cv=5, scoring='accuracy')

In [61]:
grid_dtc.fit(X_train, Y_train.toarray())

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=2021),
             param_grid={'max_depth': [2, 3, 4, 5, 6],
                         'min_samples_split': [2, 3, 4]},
             scoring='accuracy')

In [62]:
estimator = grid_dtc.best_estimator_
pred = grid_dtc.predict(X_test)

In [ ]:
pred.shape

In [64]:
accuracy_score(Y_test.toarray(),pred)

0.9